# UCU 2024 - Proyecto 1
Web Scraping - Proyecto para la asignatura Programación para Análisis de Datos

In [87]:
import requests
from bs4 import BeautifulSoup
import json

def get_info_prop(ubicacion):
    url = f'https://www.infocasas.com.uy/venta/{ubicacion}'
    
    # Simular un navegador real con headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
    
    response = requests.get(url, headers=headers)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code != 200:
        print(f"Error al acceder a la página: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    propiedades = soup.find_all('div', class_='listingCard')
    propiedades_data = []

    for propiedad in propiedades:
        # Solicitamos la información del precio
        precio_elem = propiedad.find('div', class_='lc-price')
        precio = precio_elem.get_text(strip=True) if precio_elem else 'N/A'
        
        # Solicitamos info del tamaño y habitaciones
        detalles_elem = propiedad.find('div', class_='lc-typologyTag')
        if detalles_elem:
            detalles = detalles_elem.get_text(strip=True).split(' ')
            habitaciones = detalles[0] if len(detalles) > 0 else 'N/A'
            tamano = detalles[-2] if 'm²' in detalles else 'N/A'
        else:
            habitaciones, tamano = 'N/A', 'N/A'
        
        # Solicitamos el link a la propiedad
        link_elem = propiedad.find('a', class_='lc-cardCover')
        link = f"https://www.infocasas.com.uy{link_elem['href']}" if link_elem else 'N/A'           
        
        # Solicitamos la ubicación
        ubicacion_elem = propiedad.find('strong', class_='lc-location')
        ubicacion_real = ubicacion_elem.get_text(strip=True) if ubicacion_elem else 'N/A'
        
        # Verifica si la ubicación de la propiedad coincide con la ciudad que estamos consultando
        if ubicacion.capitalize() in ubicacion_real: 
            # Guardamos los datos de cada propiedad en un diccionario
            propiedades_data.append({
                'precio': precio,
                'tamaño': tamano,
                'habitaciones': habitaciones,
                'link': link,
            })

    return propiedades_data

# Lista de departamentos
departamentos = ['montevideo', 'salto', 'maldonado']

# Lista para almacenar las ciudades y sus propiedades
ciudades_data = []

# Bucle para obtener datos de cada ciudad
for ubicacion in departamentos:
    propiedades = get_info_prop(ubicacion)
    if propiedades:
        ciudades_data.append({
            'nombre': ubicacion.capitalize(),
            'propiedades': propiedades
        })

# Estructura final del JSON con una clave "ciudades"
final_data = {
    'ciudades': ciudades_data
}

# Guardar los datos en un archivo JSON
with open('propiedades.json', 'w', encoding='utf-8') as f:
    json.dump(final_data, f, indent=4, ensure_ascii=False)

print('Datos guardados en propiedades.json')

Datos guardados en propiedades.json
